# How to use Guardrails? and Structured output

We will implement a Guardrail to check if Name is present in the user_prompt, if so, it should fail otherwise pass and return structured output

1. Input Guardrail
2. Output Gaurdrail

| Guardrail Type   | Purpose               | Example Trigger                 |
| ---------------- | --------------------- | ------------------------------- |
| Input Guardrail  | Before agent responds | User input contains banned name |
| Output Guardrail | After agent responds  | Agent response includes a name  |


In [172]:
import os
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, OpenAIChatCompletionsModel, input_guardrail, output_guardrail, GuardrailFunctionOutput, InputGuardrailTripwireTriggered, OutputGuardrailTripwireTriggered
from pydantic import BaseModel

load_dotenv(override=True)

gemini_client = AsyncOpenAI(api_key=os.getenv("GEMINI_API_KEY"), base_url=os.getenv("GEMINI_BASE_URL"))

gemini_model_name = "gemini-2.0-flash"

gemini_model = OpenAIChatCompletionsModel(openai_client=gemini_client, model=gemini_model_name)

In [149]:
class TestGuardrailResponse(BaseModel):
    is_name_present: bool
    name: str
    text: str

In [113]:
class NameCheckGuardrailResponse(BaseModel):    
    is_name_present: bool
    name: str

In [114]:
class GeminiResponseFormat(BaseModel):
    is_name_present: bool
    name: str
    # Note: Below does not work with Gemini Model but works with OpenAI Model 
    # Error: BadRequestError: Error code: 400 
    # - [{'error': {'code': 400, 'message': "Unable to submit request because one or more response schemas didn't specify the schema type field.
    
    # name: str | None = None

In [ ]:
name_check_guardrail_instructions = "You are a guardrail agent that checks if the name is present in the user prompt. If it is, it should fail otherwise pass and return structured output."
name_check_guardrail_agent = Agent(name="name_check_guardrail_agent", instructions=name_check_guardrail_instructions, model=gemini_model, output_type=NameCheckGuardrailResponse)

name_check_guardrail_agent

# Input Guardrail

In [158]:
@input_guardrail()
async def name_check_guardrail_function(ctx, agent, user_prompt):
    response = await Runner.run(name_check_guardrail_agent, user_prompt, context=ctx.context)
    is_name_present_in_prompt = response.final_output.is_name_present
    return GuardrailFunctionOutput(
        tripwire_triggered=is_name_present_in_prompt,
        output_info={"found_name": response.final_output.name}
    )


# print(name_check_guardrail_function)

In [151]:
test_gemini_model_agent = Agent(name="test_gemini_model_agent", instructions="You are a test agent that uses the Gemini model to generate a response.", model=gemini_model, output_type=TestGuardrailResponse, input_guardrails=[name_check_guardrail_function])

user_prompt_with_name = "Is Sundar Pichai your CEO?"
user_prompt_without_name = "Is AI your CEO?"

### Below will fail with `InputGuardrailTripwireTriggered: Guardrail InputGuardrail triggered tripwire` error

In [ ]:
# Below will fail with `InputGuardrailTripwireTriggered: Guardrail InputGuardrail triggered tripwire` error

response_from_test_gemini_model_agent = await Runner.run(test_gemini_model_agent, user_prompt_with_name)

print(response_from_test_gemini_model_agent.final_output)

### Customize Response Instead of Crashing with InputGuardrailTripwireTriggered

In [156]:
# But Input Guardrail returns output_info as a dictionary, so we should error message instead of error stacktrace
# Customize Response Instead of Crashing

try:
    user_prompt = "Is Sundar Pichai your CEO?"
    response_from_test_gemini_model_agent = await Runner.run(test_gemini_model_agent, user_prompt)
    print(response_from_test_gemini_model_agent.final_output)
except InputGuardrailTripwireTriggered as e:
    name_found_in_prompt = e.guardrail_result.output.output_info["found_name"]
    print(f"ERROR: Sorry, you can't mention personal names like '{name_found_in_prompt}' in this assistant.")

ERROR: Sorry, you can't mention personal names like 'Sundar Pichai' in this assistant.


In [155]:
# Working user_prompt which doesn't have name in it

response_from_test_gemini_model_agent = await Runner.run(test_gemini_model_agent, user_prompt_without_name)

print(response_from_test_gemini_model_agent.final_output)

is_name_present=False name='AI' text='I am a large language model, trained by Google. I am developed to be informative and comprehensive. I am trained on a massive amount of text data, and I am able to communicate and generate human-like text in response to a wide range of prompts and questions. However, I do not have a CEO.'


# Output Gaurdrail

In [165]:
class ProfanityCheckGuardrailResponse(BaseModel):
    is_profanity_present: bool
    profanity: str
    text: str

In [166]:
profanity_check_guardrail_instructions = "You are a guardrail agent that checks if the profanity is present in the user prompt. If it is, it should fail otherwise pass and return structured output."
profanity_check_guardrail_agent = Agent(
    name="profanity_check_guardrail_agent", 
    instructions=profanity_check_guardrail_instructions, 
    model=gemini_model, 
    output_type=ProfanityCheckGuardrailResponse
)

print(profanity_check_guardrail_agent)

Agent(name='profanity_check_guardrail_agent', handoff_description=None, tools=[], mcp_servers=[], mcp_config={}, instructions='You are a guardrail agent that checks if the profanity is present in the user prompt. If it is, it should fail otherwise pass and return structured output.', prompt=None, handoffs=[], model=<agents.models.openai_chatcompletions.OpenAIChatCompletionsModel object at 0x116494080>, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, response_include=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), input_guardrails=[], output_guardrails=[], output_type=<class '__main__.ProfanityCheckGuardrailResponse'>, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)


In [167]:
@output_guardrail()
async def profanity_check_guardrail_function(ctx, agent, user_prompt):
    response = await Runner.run(profanity_check_guardrail_agent, user_prompt, context=ctx.context)
    is_profanity_present_in_response = response.final_output.is_profanity_present
    return GuardrailFunctionOutput(
        tripwire_triggered=is_profanity_present_in_response, 
        output_info={"found_profanity": response.final_output.profanity}
    )

### Below will fail with `OutputGuardrailTripwireTriggered: Guardrail OutputGuardrail triggered tripwire` error

In [179]:
# Below will fail with `OutputGuardrailTripwireTriggered: Guardrail OutputGuardrail triggered tripwire` error

gemini_returns_profanity_agent = Agent(
    name="gemini_returns_profanity_agent",
    instructions="You are an assistant that returns profanity in the response all the time.",
    model=gemini_model,
    output_guardrails=[profanity_check_guardrail_function]
)

user_prompt = "I want to respond to someone who is a jerk so that they can feel bad about themselves."

response = await Runner.run(gemini_returns_profanity_agent, user_prompt)

print(response.final_output)

OutputGuardrailTripwireTriggered: Guardrail OutputGuardrail triggered tripwire

### Customize Response Instead of Crashing with OutputGuardrailTripwireTriggered

In [182]:
try:
    response = await Runner.run(gemini_returns_profanity_agent, user_prompt)
    print(response.final_output)
except OutputGuardrailTripwireTriggered as e:
    found_profanity_in_agents_response = e.guardrail_result.output.output_info["found_profanity"]
    print(f"ERROR: Agent responded with profanity: {found_profanity_in_agents_response}")

ERROR: Agent responded with profanity: fuck
